## Pré-requis

- Créer un environnement virtuel, ensuite l'activer
- Lancer jupyter lab
- Créer un notebook nommé `sqlalchemy-db`
- enfin installer, a minima, les libraires `pandas`, `sqlalchemy` et `sqlalchemy-utils`

## Installer SQLAlchemy

In [11]:
# !pip install sqlalchemy

!pip install sqlalchemy-utils

  Using cached SQLAlchemy_Utils-0.41.1-py3-none-any.whl (92 kB)


## Importer les modules
Importez les modules nécessaires de SQLAlchemy dans votre script Python :

In [9]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date
from sqlalchemy.inspection import inspect
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy_utils import database_exists, create_database

## Créer une connexion à la base de données

Créez une connexion à la base de données en utilisant create_engine et spécifiez l'URL de connexion appropriée pour votre base de données :

In [2]:
# SQLite en mémoire (exemple)
engine = create_engine('sqlite:///:memory:')

if not database_exists(engine.url):
    create_database(engine.url)

In [4]:
engine.url

sqlite:///:memory:

### SQLite: Création DB

### Déclarer un modèle

Définissez une classe modèle en utilisant declarative_base et définissez des colonnes avec des types de données appropriés :

In [3]:
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    user_id = Column(Integer, primary_key=True)
    name = Column(String, comment="Nom & Prenom de l'utilisateur", nullable=False)
    age = Column(Integer)
    fil_id = Column(Integer)
    # promo = Column(String)
    
class Filiere(Base):
    __tablename__ = 'filiere'

    fil_id = Column(Integer, primary_key=True)
    name = Column(String, comment="Nom & Prenom de la filière", nullable=False)
    date_creation = Column(Date)
    responsable = Column(String)


### Créer les tables

Créez les tables dans la base de données en utilisant Base.metadata.create_all(engine) :

In [4]:
# creation des tables associées à Base
Base.metadata.create_all(engine, checkfirst=True)

In [5]:
db_insp = inspect(engine)

In [6]:
db_insp.get_table_names()

['users']

In [10]:
db_insp.get_columns(table_name="users")

[{'name': 'user_id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'primary_key': 1},
 {'name': 'name',
  'type': VARCHAR(),
  'nullable': False,
  'default': None,
  'primary_key': 0},
 {'name': 'age',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'promo',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'primary_key': 0}]

In [14]:
# Créer une session : Créez une session pour interagir avec la base de données en utilisant sessionmaker 

Session = sessionmaker(bind=engine)
session = Session()

In [23]:
# Ajouter un nouvel utilisateur
user = User(name='John Doe', age=30, promo="DIT")
session.add(user)
session.commit()

### Autres bases de données

Pour d'autres bases de données, telle que MySQL ou PostgreSQL, vous devrez spécifier l'URL de connexion correspondante, par exemple :

In [16]:
User.__tablename__

'users'

In [58]:
users = session.query(User).all()

In [59]:
len(users)

1

In [60]:
for user in users:
    print(user.user_id, user.name, user.age)

1 John Doe 40


In [51]:
# Mettre à jour un utilisateur
users_filter = session.query(User).filter_by(age='30', name="John Doe")
users_filter

In [52]:
for user in users_filter:
    print(user.user_id, user.name)

In [53]:
# Mettre à jour un utilisateur
user = session.query(User).filter_by(name='John Doe').first()
user.age = 40
session.commit()

In [57]:
# Supprimer un utilisateur
user = session.query(User).filter_by(name='John Doe', age="30").first()
session.delete(user)
session.commit()

#### Explorez vos données avec Pandas

## TP

- **Missions**
Vous êtes Data Engineer au Crédit Corporatif, une banque coopérative qui propose différents produits d’épargne responsable.

Le service marketing a passé une commande de création de base de données permettant de suivre la performance des campagnes marketing.

Le service marketing de la banque lance des campagnes marketing. Une campagne consiste à faire de la publicité pour les produits d’épargne de la banque. Ces publicités (appelées bannières publicitaires) s’affichent sur différents sites du web (chaque affichage sur un site quelconque est appelé une impression). Les internautes peuvent cliquer sur ces publicités, ils sont alors redirigés vers le site de la banque.


- **Données**

Les données sont disponibles [ici](https://github.com/MouslyDiaw/creez-dashboard/tree/main/data)

Les tables à créer sont:

    - impressions

    Chaque ligne correspond à une impression, c’est-à-dire à un affichage d’une publicité sur un site donné, et sur l’écran d’un utilisateur donné.

    Il contient 4 colonnes :
    timestamp : c’est une date codée, qui est la date d’affichage de la publicité.
    cookie_id permet d’identifier l’utilisateur qui a vu la publicité, chaque publicité étant liée à une campagne précise.
    campaign_id est l’identifiant de la campagne de marketing.
    external_site_id indique le site sur lequel la publicité s’est affichée

    Cette table contient 29 758 lignes

    - clics

    À la suite de l’affichage de ces publicités, certains utilisateurs cliquent dessus. La table clics enregistre la date et l’heure du clic dans timestamp, et l’utilisateur est repéré grâce à cookie_id.


    - achats

    Enfin, la table achats indique chaque achat réalisé par un utilisateur ayant cliqué sur une publicité. On y retrouve le timestamp, l’identifiant utilisateur cookie_id, le produit acheté product_id, son genre gender, son département de résidence dept, le prix d’achat price et son âge (age).
    
    
    
    
**Demandes d'analyse**: répondre aux questions suivantes:
- Quel est le chiffre d'affaires généré par Campagne ?
- Quel est le nombre d’affichages des bannières ?
- Combien de personnes viennent sur le site web (nombre de clics) ?
- Combien de personnes souscrivent aux offres ?
- autres analyses permettant de comprendre les performances clés d'une campagne


envoyer au plus tard le vendredi 16/06/2023 à 18h avec l'objet `TP - sqlalchemy`